## DRAGON: A Comprehensive Convolutional Neural Networked Designed to Discover Dual AGN, Offset AGN, and Galaxy Merger Candidates in Large Scale Survey Fields.

### Authors: Isaac Moskowitz & Jeremy Ng
### Collaborators: C. Megan Urry
### Started 5/29/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from astropy.io import fits
from astropy.table import Table

2024-05-29 09:41:16.500580: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-29 09:41:17.304918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 09:41:17.304952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 09:41:17.310337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 09:41:17.673572: I tensorflow/core/platform/cpu_feature_g

The following script is used to display the `FITS` table containing training data of approximately 220 Dual AGN and Offset AGN from Stemo et al (2020). 

In [7]:
with fits.open("data_preprocessing/training_datasets/offset_AGN_datasets/ACS_AGN_Merger_Catalog.fits") as hdul:
    data = hdul[1].data
    df = pd.DataFrame(data.byteswap().newbyteorder())
print(df)

IndentationError: unexpected indent (1263819810.py, line 3)

#### Composite Offset AGN Dataset Creator
The following functions will create training datasets for the "Offset AGN" catagory for `DRAGON`. These images are typically low redshift $z < 1.0$. Since we do not have access to sufficiently many confirmed offset AGN to train the model on, we create additional versions in much the same way as we created dual AGN candidates. First, we select several (~$100$) single galaxies with varying orientation and structure imaged with HSC. Then, we select a confirmed AGN at a comparable redshift (preferably the same redshift, though if $z$ is within $0.1$ from the redshift of the galaxy we can make that work) and request cutouts from HSC such that the AGN is offset from the image center by a specified angular separation (in arcseconds) and rotation angle (in degrees). Once downloaded, we "stitch" these images together to create composite offset AGN images that resemble real data in both noise profile and feature structure. 

In [ ]:
def imgdwnldr_gen(args):
    
    df,num_start,num_stop,fltr = args
    
    t = Table()
    t["rerun"] = ["pdr3_wide"]*len(df[num_start:num_stop])
    t["filter"] = [fltr]*len(df[num_start:num_stop])
    #t["tract"] = df["tract"][num_start:num_stop]
    t["ra"] = df["ra"][num_start:num_stop]
    t["dec"] = df["dec"][num_start:num_stop]
    t["type"] = ["coadd"]*len(df[num_start:num_stop])
    t["sh"] = ["8asec"]*len(df[num_start:num_stop])
    t["sw"] = ["8asec"]*len(df[num_start:num_stop])
    t["name"] = df["name"][num_start:num_stop]
                  
    return t
def table_writer(args):
    
    table, i, filepath_prefix, overwrite, tableformat, comment= args
    if not exists(filepath_prefix):
        os.makedirs(filepath_prefix)
    
    table.write(filepath_prefix + "_" + ".txt",
                overwrite=overwrite,format=tableformat,
                comment=comment)
#args = (test2DF, 0, 35, "HSC-R")
#secondTestTable = imgdwnldr_gen(args)
#filepath_prefix = 'input_second_test_HSC_new'
def write_dwnldr_files(df,step,filepath_prefix,fltr,write=True,
                      overwrite=True): 
    #if not exists(filepath_prefix):
        #os.makedirs(filepath_prefix)
    len_df = len(df)
    iters = int(len_df/step)
    
    lwr_ends = range(0,len_df,step)
    upr_ends = range(step,len_df+step,step)
    fltrs = [fltr]*(iters+1)
    dfs = [df]*(iters+1)
    
    args_imgdwnldr_gen = list(zip(dfs,lwr_ends,upr_ends,fltrs))
    #args_imgdwnldr_gen = (dfs,lwr_ends,upr_ends,fltrs)
    #imgdwnldr_gen(args_imgdwnldr_gen)
    with mp.Pool() as pool:
        tables = list(tqdm(pool.imap(imgdwnldr_gen,args_imgdwnldr_gen), total = iters+1))
    
    if write is True:
        paths = [filepath_prefix]*(iters+1)
        overwrites = [overwrite]*(iters+1)
        tableformats = ['ascii.commented_header']*(iters+1)
        comments = ["#?"]*(iters+1)
        i_s = np.arange(iters+1)
    
        args_table_writer = list(zip(tables,i_s,paths,overwrites,tableformats,comments))
        #args_table_writer = (tables,i_s,paths,overwrites,tableformats,comments)
        #table_writer(args_table_writer)
        
        with mp.Pool() as pool:
            exit_codes = list(tqdm(pool.imap(table_writer,args_table_writer), total = iters+1))
        print("got here")
        
    else:
        for table in tables:
            print(table)
def write_rotation_test_coordinates(quasarName, separation, raStart, decStart, fltr):
    count = 0
    degree_separation = separation/3600
    raList = []
    decList = []
    quasarNameList = []
    for i in np.arange(0, 2*np.pi, np.pi/18):
        ra = raStart + degree_separation* np.cos(i)
        dec = decStart + degree_separation* np.sin(i)
        raList.append(ra)
        decList.append(dec)
        quasarNameNew = quasarName + str(fltr) + "_rotated_" + str(count) + "_degrees"
        print(quasarNameNew)
        quasarNameList.append(quasarNameNew)
        count+=10
    return raList, decList, quasarNameList

In [1]:
#!pwd
"""/Users/moskowitzi/Library/CloudStorage/Dropbox/First_Year_at_Yale/Summer_2022/Quasar_Research/Quasar_Images/Spring_Equitorial_HSC_Band/spring_equitorial_dowloaded_images"""
os.chdir("/vast/palmer/scratch/urry/iam37/")
#All Spring Equitorial Quasars in SDSS and HSC
starttime = time.time()
fits_table_dr16q = 'DR16Q_v4.fits'
hdu1 = fits.open(fits_table_dr16q)
data = hdu1[1].data
min_row_fall_1 = 668635
max_row_fall_1 = 750414
min_row_fall_2 = 1
max_row_fall_2 = 116727
quasarNameList_fall = []
quasarRAList_fall = []
quasarDecList_fall = []
quasarZList_fall = []

#Adding lists for known galaxies
galaxy_name_list = []
galaxy_ra_list = []
galaxy_dec_list = []
galaxy_z_list = []
"""Fall Equatorial Band"""
for j in range(min_row_fall_1, max_row_fall_2):
    if(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and (data[j].field(6) == data[j].field(7) == "QSO" or data[j].field(6) == "UNK" or data[j].field(7) == 'UNK')):
        quasarName = data[j].field(6) + " " + data[j].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        quasarNameNoSpaceNoPeriod = quasarNameNoSpace.replace(".", "_")
        quasarNameNoPlus = quasarNameNoSpaceNoPeriod.replace("+", "_")
        inputQuasarName = quasarNameNoPlus.replace("-", "_")
        quasarNameList_fall.append(inputQuasarName)
        quasarRAList_fall.append(data[j].field(1))
        quasarDecList_fall.append(data[j].field(2))
        quasarZList_fall.append(data[j].field(28))
    elif(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and data[j].field(6) == data[j].field(7) == 'GALAXY'):
        galaxy_name = data[j].field(6) + "_" + data[j].field(0)
        galaxy_name_list.append(galaxy_name)
        galaxy_ra_list.append(data[j].field(1))
        galaxy_dec_list.append(data[j].field(2))
        galaxy_z_list.append(data[j].field(28))
for j in range(min_row_fall_2, max_row_fall_2):
    if(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and (data[j].field(6) == data[j].field(7) == "QSO" or data[j].field(6) == 'UNK' or data[j].field(7) == "UNK")):
        quasarName = data[j].field(6) + " " + data[j].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        quasarNameNoSpaceNoPeriod = quasarNameNoSpace.replace(".", "_")
        quasarNameNoPlus = quasarNameNoSpaceNoPeriod.replace("+", "_")
        inputQuasarName = quasarNameNoPlus.replace("-", "_")
        quasarNameList_fall.append(inputQuasarName)
        quasarRAList_fall.append(data[j].field(1))
        quasarDecList_fall.append(data[j].field(2))
        quasarZList_fall.append(data[j].field(28))
    elif(data[j].field(2) >= -1.0 and data[j].field(2) <= 7.0 and data[j].field(6) == data[j].field(7) == 'GALAXY'):
        galaxy_name = data[j].field(6) + "_" + data[j].field(0)
        galaxy_name_list.append(galaxy_name)
        galaxy_ra_list.append(data[j].field(1))
        galaxy_dec_list.append(data[j].field(2))
        galaxy_z_list.append(data[j].field(28))
quasarNameList_fall = np.asarray(quasarNameList_fall)
for j in range(len(quasarNameList_fall)):
    if quasarNameList_fall[j].find("QSO") == -1:
        print(quasarNameList_fall[j])
        print(j)
else:
    print("All QSO objects for Fall Equatorial")
print(len(quasarNameList_fall))
print(len(quasarDecList_fall))
header = ['ra', 'dec', 'Quasar name', 'Z']
#print(len(quasarNameList))
if not exists("HSC_survey_bands/"):
    os.makedirs("HSC_survey_bands/")
with open("HSC_survey_bands/fall_equitorial_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_fall)):
        combinedData = [quasarRAList_fall[i], quasarDecList_fall[i], quasarNameList_fall[i], quasarZList_fall[i]]
        writer.writerow(combinedData)
fall_df = pd.read_csv("HSC_survey_bands/fall_equitorial_quasar_list.csv", delimiter=',')
#print(fall_df)
"""Spring Equatorial Band"""
valid_rows1 = data[170000:552181]
print(data[0][0])
print(data[0].field(2))
quasarNameList_spring = []
quasarRAList = []
quasarDecList = []
quasarZList = []
#We know that HSC's field contains RA = 130º at 199012, I will now test smaller RA's to see if they are also contained within the 
#survey field
for i in range(191617, 552181):
    if (data[i].field(2)>= -7.0 and data[i].field(2) <= -1.0 and (data[j].field(6) == data[j].field(7) == "QSO" or data[j].field(6) == 'UNK' or data[j].field(7) == "UNK")):
        quasarName = data[i].field(6) + " " + data[i].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        quasarNameNoSpaceNoPeriod = quasarNameNoSpace.replace(".", "_")
        quasarNameNoPlus = quasarNameNoSpaceNoPeriod.replace("+", "_")
        inputQuasarName = quasarNameNoPlus.replace("-", "_")
        quasarNameList_spring.append(inputQuasarName)
        quasarRAList.append(data[i].field(1))
        quasarDecList.append(data[i].field(2))
        quasarZList.append(data[i].field(28))
    elif(data[i].field(2) >= -7.0 and data[i].field(2) <= -1.0 and data[i].field(6) == data[i].field(7) == 'GALAXY'):
        galaxy_name = data[i].field(6) + "_" + data[i].field(0)
        galaxy_name_list.append(galaxy_name)
        galaxy_ra_list.append(data[i].field(1))
        galaxy_dec_list.append(data[i].field(2))
        galaxy_z_list.append(data[i].field(28))
quasarNameList_spring = np.asarray(quasarNameList_spring)
for j in range(len(quasarNameList_spring)):
    if quasarNameList_spring[j].find("QSO") == -1:
        print(quasarNameList_spring[j])
        print("Not a QSO Object")
        print(j)
else:
    print("All QSO objects")
#print(len(quasarNameList))
header = ['ra', 'dec', 'Quasar name']
print(len(quasarNameList_spring))
with open("HSC_survey_bands/spring_equitorial_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_spring)):
        combinedData = [quasarRAList[i], quasarDecList[i], quasarNameList_spring[i], quasarZList[i]]
        writer.writerow(combinedData)
spring_df = pd.read_csv("HSC_survey_bands/spring_equitorial_quasar_list.csv", delimiter=',')

#In this section of the download, we are working in the Spring Equitorial survey path, which goes from roughly
#120 degrees to 225 degrees of RA and -7 to 1 degrees of Dec
"""Northern Sky Band"""
min_row_north = 448834
max_row_north = 602111
quasarNameList_north = []
quasarRAList_north = []
quasarDecList_north = []
quasarZList_north = []
for k in range(min_row_north, max_row_north):
    if(data[k].field(2) >= 42.5 and data[k].field(2) <= 44.0 and (data[k].field(6) == data[k].field(7) == "QSO" or data[k].field(6) == 'UNK' or data[k].field(7) == "UNK")):
        quasarName = data[k].field(6) + " " + data[k].field(0)
        quasarNameNoSpace = quasarName.replace(" ", "_")
        quasarNameNoSpaceNoPeriod = quasarNameNoSpace.replace(".", "_")
        quasarNameNoPlus = quasarNameNoSpaceNoPeriod.replace("+", "_")
        inputQuasarName = quasarNameNoPlus.replace("-", "_")
        quasarNameList_north.append(inputQuasarName)
        quasarRAList_north.append(data[k].field(1))
        quasarDecList_north.append(data[k].field(2))
        quasarZList_north.append(data[k].field(28))
    elif(data[k].field(2) >= 42.5 and data[k].field(2) <= 44.0 and data[k].field(6) == data[k].field(7) == 'GALAXY'):
        galaxy_name = data[k].field(6) + "_" + data[k].field(0)
        galaxy_name_list.append(galaxy_name)
        galaxy_ra_list.append(data[k].field(1))
        galaxy_dec_list.append(data[k].field(2))
        galaxy_z_list.append(data[k].field(28))
quasarNameList_north = np.asarray(quasarNameList_north)
for k in range(len(quasarNameList_north)):
    if quasarNameList_north[k].find("QSO") == -1:
        print(quasarNameList_north[k])
        print(k)
else:
    print("All QSO objects for Northern Sky")
print(len(quasarNameList_north))
print("Total number of QSO objects = " + str(len(quasarNameList_fall) + len(quasarNameList_spring) + len(quasarNameList_north)))
with open("HSC_survey_bands/northern_sky_quasar_list.csv", 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0, len(quasarNameList_north)):
        combinedData = [quasarRAList_north[i], quasarDecList_north[i], quasarNameList_north[i], quasarZList_north[i]]
        writer.writerow(combinedData)
northern_df = pd.read_csv("HSC_survey_bands/northern_sky_quasar_list.csv", delimiter=',')
endtime = time.time()
print("--- %s seconds ---" % (endtime - starttime))

NameError: name 'os' is not defined